
# Pytorch Forecasting Code


Our Example of Pytorch Forecasting, specifically their TemporalFusionTransformer

In [9]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
# import dataset, network to train and metric to optimize
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [10]:
# pytorch_forecasting.metrics.RMSE

In [11]:
import pandas as pd
df = pd.read_csv("scpppi/processed_data_l.csv")

In [12]:

def clean_df(df):
    df = df.drop(columns=["Unnamed: 0","datetime","granule_id"])
    df["datetime_dt"] = pd.to_datetime(df["datetime_dt"] )
    df["month"] = [i.month for i in df["datetime_dt"]]
    df["month"] = df["month"].astype(str).astype("category")
    one_hot = pd.get_dummies(df["month"])
    df = df.join(one_hot)
    df = df.drop('month',axis = 1)
    df = df.drop('date',axis = 1)
#     one_hot = pd.get_dummies(df["location"])
#     df = df.join(one_hot)
#     one_hot = pd.get_dummies(df["grid_id"])
#     df = df.join(one_hot)
#     df = df.drop('grid_id',axis = 1)
    return df

In [13]:
df_cleaned = clean_df(df)

In [14]:
df_cleaned.head()

,grid_id,value,datetime_dt,location,AOD at 0.47 micron_0,AOD at 0.47 micron_1,AOD at 0.47 micron_2,AOD at 0.47 micron_3,AOD at 0.47 micron_4,AOD at 0.47 micron_5,...,11,12,2,3,4,5,6,7,8,9
0,3S31A,11.4,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
1,A2FBI,17.0,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
2,DJN0F,11.1,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
3,E5P9N,22.1,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
4,FRITQ,29.8,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0


In [15]:
data = df_cleaned

In [16]:
data["time_idx"] = data["datetime_dt"].dt.year * 365 + data["datetime_dt"].dt.month *30 + data["datetime_dt"].dt.day
data["time_idx"] -= data["time_idx"].min()

In [17]:
max_prediction_length = 1
max_encoder_length = 18
training_cutoff = data["time_idx"].max() - max_prediction_length


In [18]:
time_varying_known_reals = [i.replace(".","_") for i in list(data.columns[4:])]
time_varying_known_reals[0:2]

['AOD at 0_47 micron_0', 'AOD at 0_47 micron_1']

In [19]:
time_varying_known_reals

['AOD at 0_47 micron_0',
 'AOD at 0_47 micron_1',
 'AOD at 0_47 micron_2',
 'AOD at 0_47 micron_3',
 'AOD at 0_47 micron_4',
 'AOD at 0_47 micron_5',
 'AOD at 0_47 micron_6',
 'AOD at 0_47 micron_7',
 'AOD at 0_47 micron_8',
 'AOD at 0_47 micron_9',
 'AOD at 0_47 micron_10',
 'AOD at 0_47 micron_11',
 'AOD at 0_47 micron_12',
 'AOD at 0_47 micron_13',
 'AOD at 0_47 micron_14',
 'AOD at 0_47 micron_15',
 'AOD at 0_47 micron_16',
 'AOD at 0_47 micron_17',
 'AOD at 0_47 micron_18',
 'AOD at 0_47 micron_19',
 'AOD at 0_47 micron_20',
 'AOD at 0_47 micron_21',
 'AOD at 0_47 micron_22',
 'AOD at 0_47 micron_23',
 'AOD at 0_47 micron_24',
 'AOD at 0_47 micron_25',
 'AOD at 0_47 micron_26',
 'AOD at 0_47 micron_27',
 'AOD at 0_47 micron_28',
 'AOD at 0_47 micron_29',
 'AOD at 0_47 micron_30',
 'AOD at 0_47 micron_31',
 'AOD at 0_47 micron_32',
 'AOD at 0_47 micron_33',
 'AOD at 0_47 micron_34',
 'AOD at 0_47 micron_35',
 'AOD at 0_47 micron_36',
 'AOD at 0_47 micron_37',
 'AOD at 0_47 micron_3

In [20]:
data.columns = list(data.columns)[:4]+ time_varying_known_reals

In [21]:
data


,grid_id,value,datetime_dt,location,AOD at 0_47 micron_0,AOD at 0_47 micron_1,AOD at 0_47 micron_2,AOD at 0_47 micron_3,AOD at 0_47 micron_4,AOD at 0_47 micron_5,...,12,2,3,4,5,6,7,8,9,time_idx
0,3S31A,11.400000,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,1,0,0,0,0,0,0,0,0
1,A2FBI,17.000000,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,1,0,0,0,0,0,0,0,0
2,DJN0F,11.100000,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,1,0,0,0,0,0,0,0,0
3,E5P9N,22.100000,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,1,0,0,0,0,0,0,0,0
4,FRITQ,29.800000,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39445,GVQXS,99.512500,2018-04-17 18:30:00+00:00,Delhi,-0.558915,0.114815,-0.115747,0.257394,-0.049393,0.243800,...,0,0,0,1,0,0,0,0,0,76
39446,GVQXS,101.068421,2018-04-18 18:30:00+00:00,Delhi,-0.559187,0.114805,-0.115720,0.257316,-0.049393,0.243815,...,0,0,0,1,0,0,0,0,0,77
39447,GVQXS,157.216667,2018-04-19 18:30:00+00:00,Delhi,-0.559288,0.114813,-0.115717,0.257326,-0.049380,0.243832,...,0,0,0,1,0,0,0,0,0,78
39448,GVQXS,149.870833,2018-04-25 18:30:00+00:00,Delhi,-0.559169,0.114806,-0.115722,0.257317,-0.049391,0.243812,...,0,0,0,1,0,0,0,0,0,84


In [22]:
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="value",
    group_ids=["location","grid_id"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=["location","grid_id"],
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "value"
    ],
    target_normalizer=GroupNormalizer(
        groups=[ "location","grid_id"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

In [23]:
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=16)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=16)

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1238: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 2 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__location': 'Delhi', '__group_id__grid_id': '7F1D1'}, {'__group_id__location': 'Delhi', '__group_id__grid_id': 'WZNCR'}]
  warnings.warn(


In [24]:
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)


80.87256622314453

In [25]:
from torch.nn import MSELoss

In [26]:

from torchmetrics import MeanSquaredError


In [27]:
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=1,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    attention_head_size=2,
    dropout=0.2,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=1,  
    loss=RMSE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Number of parameters in network: 2852.3k


In [28]:
import torch.nn.functional as F

In [ ]:
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)


/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | RMSE                            | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 819   
3  | prescalers                         | ModuleDict                      | 33.1 K
4  | static_variable_selection          | VariableSelectionNetwork        | 1.7 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1.4 M 
6  | de

Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 52. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
Global seed set to 42


Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/sit